In [1]:
from cfg import CLIENTE_ID, CLIENTE_SECRET, SPOTIPY_REDIRECT_URI
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

POR las dudas aclaro, no puedo proporcionar el archivo py "cfg" ya que son datos propios que no puedo compartir, espero que esto no sea un problema

In [134]:
scope='user-read-recently-played'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENTE_ID,
                                               client_secret=CLIENTE_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope=scope))

In [135]:
def extract(date, limit=50):

    ds = int(date.timestamp()) * 1000
    return sp.current_user_recently_played(limit=limit, after=ds)

In [136]:
def transform(raw_data, date):
    datas = []

    for r in raw_data['items']:
        datas.append({   
                'played_at': r["played_at"],
                'artist': r['track']['artists'][0]['name'],
                'track' : r['track']['name']})
    data = pd.DataFrame(datas)
    return data

In [2]:
###Conexion Redshift###
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws"
data_base="data-engineer-database"
port='5439'
user="pazmartinexe_coderhouse"
with open("C:/Users/Windows/Downloads/pass.txt",'r') as f:
    pwd=f.read()

try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [140]:

 ###Insertar Datos##
def cargar_en_redshift(conn, table_name, dataframe):
        dtypes= data.dtypes
        cols= list(dtypes.index )
        tipos= list(dtypes.values)
        type_map = {'object': 'VARCHAR(100)','object': 'VARCHAR(30)','object': 'VARCHAR(100)'}
        sql_dtypes = [type_map[str(dtype)] for dtype in tipos]
        
       # Combine column definitions into the CREATE TABLE statement
        column_defs = [f"{name} {data_type}" for name, data_type in zip(cols, sql_dtypes)]
        table_schema = f"""
            DROP TABLE IF EXISTS {table_name};
            CREATE TABLE IF NOT EXISTS {table_name} (
                {', '.join(column_defs)}
            );
            """
        #Crear la tabla
        cur = conn.cursor()
        cur.execute(table_schema)
    # Generar los valores a insertar
        values = [tuple(x) for x in data.to_numpy()]
        # Definir el INSERT
        insert_sql = f"INSERT INTO musica_spotify ({', '.join(cols)}) VALUES %s"
        
      
    # Execute the transaction to insert the data
        cur.execute("BEGIN")
        execute_values(cur, insert_sql, values)
        cur.execute("COMMIT")
        print('Proceso terminado')

Comenzamos a ejecutar las funciones EXTRACT, TRANSFORM y LOAD

In [141]:
# La variables date toma el dia actual y le resta 3 dias para poder realizar la busqueda de las canciones
date = datetime.today() - timedelta(days=3)
#extract
data_raw = extract(date)
print(f"extrayendo {(data_raw['items'])} registers") 

extrayendo [{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2LRoIwlKmHjgvigdNGBHNo'}, 'href': 'https://api.spotify.com/v1/artists/2LRoIwlKmHjgvigdNGBHNo', 'id': '2LRoIwlKmHjgvigdNGBHNo', 'name': 'Feid', 'type': 'artist', 'uri': 'spotify:artist:2LRoIwlKmHjgvigdNGBHNo'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 

In [142]:
#transform
data = transform(data_raw,date)
print(f"{data.shape[0]} registers after transform")

50 registers after transform


In [143]:
#load
cargar_en_redshift(conn=conn,table_name='musica_spotify',dataframe=data)
print('Done')

['VARCHAR(100)', 'VARCHAR(100)', 'VARCHAR(100)']

           DROP TABLE IF EXISTS musica_spotify;
           CREATE TABLE IF NOT EXISTS musica_spotify (
               played_at VARCHAR(100), artist VARCHAR(100), track VARCHAR(100)
           );
           
Proceso terminado
done


In [3]:
conn.close()